In [ ]:
from constants.tf import my_pic
import numpy as np
import onnxruntime as rt
import os
from os import path
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Lambda
from tensorflow.keras import Model, Sequential, losses
import tf2onnx.convert
from typing import Tuple


In [ ]:
# constants
random_state = 112
tf.random.set_seed(random_state)

# load data
fashion_mnist = tf.keras.datasets.fashion_mnist

# output path
base_output_path = './out/models'
h5_path = path.join(base_output_path, 'fashion_mnist_simple.h5')
default_path = path.join(base_output_path, 'fashion_mnist_simple')
onnx_path = path.join(base_output_path, 'fashion_mnist_simple_tf_api.onnx')

# set number of threads used
tf.config.threading.set_intra_op_parallelism_threads(2)


In [ ]:
def flatten(np_array: np.ndarray) -> np.ndarray:
    flatten_array = []
    for image in np_array:
        flatten_array.append(image.flatten())
    return np.array(flatten_array)


def prepare_dir(dir_: str):
    if not path.isdir(dir_):
        os.makedirs(dir_)


train_set: Tuple[np.ndarray, np.ndarray]
x_test: np.ndarray
y_test: np.ndarray
x_train: np.ndarray
y_train: np.ndarray
x_val: np.ndarray
y_val: np.ndarray

prepare_dir(base_output_path)

In [ ]:
train_set, (x_test, y_test) = fashion_mnist.load_data()
x_train, x_val, y_train, y_val = train_test_split(
    train_set[0], train_set[1], test_size=0.2, random_state=random_state)

x_train = flatten(x_train)
x_val = flatten(x_val)
x_test = flatten(x_test)

In [ ]:
def train(x: np.ndarray, y: np.ndarray) -> Model:
    model = Sequential([
        Lambda(lambda x: x / 255, input_shape=(784,)),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])

    loss = losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(
        loss=loss, metrics=['accuracy'], optimizer='adam')
    model.fit(x, y, epochs=10)
    return model


def predict(model: Model, x: np.ndarray, y: np.ndarray):
    loss, acc = model.evaluate(x, y, verbose=2)


def save_onnx(model: Model):
    input_spec = (tf.TensorSpec((1, 784), tf.uint8, name='input'),)
    tf2onnx.convert.from_keras(model, input_spec, output_path=onnx_path)


In [ ]:
model = train(x_train, y_train)
predict(model, x_test, y_test)

model.save(default_path)
# h5
model.save(h5_path)

# onnx
save_onnx(model)

# Default Prediction

In [ ]:
with tf.device('/CPU:0'):
    model_default = tf.keras.models.load_model(default_path)

    prediction_h5 = model_default.predict([my_pic])
    print(prediction_h5, np.argmax(prediction_h5[0]))

In [ ]:
%timeit with tf.device('/CPU:0'): model_default.predict([my_pic])

# H5 Prediction

In [ ]:
with tf.device('/CPU:0'):
    model_h5 = tf.keras.models.load_model(h5_path)

    prediction_h5 = model_h5.predict([my_pic])
    print(prediction_h5, np.argmax(prediction_h5[0]))

In [ ]:
%timeit with tf.device('/CPU:0'): model_h5.predict([my_pic])

# ONNX Prediction

In [ ]:
sess = rt.InferenceSession(onnx_path)

# set number of threads
sess_options = sess.get_session_options()
sess_options.intra_op_num_threads = 2

input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

prediction_onnx = sess.run([output_name], {input_name: [my_pic]})
print(prediction_onnx, np.argmax(prediction_onnx))

In [ ]:
%timeit sess.run([output_name], {input_name: [my_pic]})

In [ ]:
# answer
# 0 T-shirt/top
# 1 Trouser
# 2 Pullover
# 3 Dress
# 4 Coat
# 5 Sandal
# 6 Shirt
# 7 Sneaker
# 8 Bag
# 9 Ankle boot

# Saving Speed

In [ ]:
%timeit model.save(default_path)

In [ ]:
%timeit model.save(h5_path)

In [ ]:
%timeit save_onnx(model)

# Random Prediction

In [ ]:
%timeit np.random.randint(0, 255, [1, 784], np.uint8)

In [ ]:
%timeit sess.run([output_name], {input_name: np.random.randint(0, 255, [1, 784], np.uint8)})

In [ ]:
%timeit with tf.device('/CPU:0'): model_h5.predict(np.random.randint(0, 255, [1, 784], np.uint8))